# Fitting

We're going to take a closer look at the parameters and how we can enhance this.

## What happens when we fit the model

Let's start with a linear regression.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.DataFrame({'weight': [0.7, 2.4, 2.8],
                   'height': [1.5, 1.8, 3.2]})

X = df[['weight']].values
y = df['height'].values

In [ ]:
size=[0, 4]

def draw_chart(X=X, y=y, size=size):

    plt.scatter(X, y)
    plt.xlabel('weight')
    plt.ylabel('height')
    plt.axis(size*2)
    
draw_chart()

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X, y)

In [ ]:
# a coefficient
model.coef_[0]

In [ ]:
# b coefficient
model.intercept_

In [ ]:
draw_chart()
plt.plot(size, model.predict(np.array(size).reshape(-1, 1)), color='red');

## Model prediction in a equation

What happens during `.fit()`? Any model can be expressed as: 

$y = h(X, \beta) + error$

- $h$ is called our hypothesis function
- $X$ is our input (features)
- $\beta$ is the parameters of our model
- $error$ is the difference between y true ($y$) and y pred ($\hat{y}$).
- $h(X, \beta)$ is called our prediction ($\hat{y}$)

## For a Linear Regression

For a linear regression : $h(X, \beta) = \beta_0 + \beta_1 X_1$, where $\beta_0$ is the interpect and  $\beta_0$ our coef ($a$).

When we run a ```.fit()```, the algorithm tries to find the best parameters $\beta_0$ and  $\beta_0$ in order to minimize the $error$.

## L2 Loss Function

L2 Loss Function is used to minimize the error which is the sum of the all the squared differences between the true value and the predicted value.

```.fit()``` minimizes $L(error)$
$L_{OLS} = \lVert error \rVert^2 = \lVert y - \beta_0 - \beta_1 X_1 \rVert^2$

(OLS = Ordinary Least Squares, sometimes written LS = Least Square Errors)

We often write: $\beta = \text{argmin}_\beta L(\beta, X, y, h)$

**Note:** You also have the L1 Loss (also called LAD = Least Absolute Deviations), less sensitive to outliers, but less used:
$L_{L1} = \lVert error \rVert = \lVert y - \beta_0 - \beta_1 X_1 \rVert$

## Solvers

There are different "solvers" you can use to minimize $L(\beta)$ :
- Exact mathematical resolution (matrix inversion, but often too complex)
- Iteratives approaches. We'll assign $\beta_0$ and $\beta_1$ random values and we're going to minimize the errors through multiples iterations.

In [ ]:
from sklearn.linear_model import LogisticRegression

LogisticRegression(solver='newton-cg')

Imagine that we already know the value of the ideal slope, $\beta_1 = 0.64$, and need to find the optimal  $\beta_0$.

<div>
<img src="files/height_weight_SSR_intercept.png" width="70%" align='center' source="https://github.com/MoinDalvs/Gradient_Descent_For_beginners" />
</div>

1. Let's randomly initialize an intercept, say at 0.
1. Now, let's compute the loss at that intercept value; here, the loss is the Sum of Squared Residuals (**SSR**).
1. Change the intercept and repeat the process until we find the smallest loss.

If we look at the Loss Function, we see that it has a convex shape.

<div>
<img src="files/solver_ssr_1.png" width="70%" align='center' source="https://github.com/MoinDalvs/Gradient_Descent_For_beginners" />
</div>

**Problems**:

- We could miss the exact minimum if our steps are too large
- We don't know the best  $\beta_1$ to start with.

So let's discover the most basic but very powerful iterative method: the **Gradient Descent**.

## Gradient Descent

### 1D Descent Step-by-Step

- Uses the **slope** (gradient) of the Loss Function as an indicator.
- As the slope approaches zero, the loss approaches its minimum.

$\frac{\delta Loss Function}{\delta Parameter}$

The slope is equal to the partial derivative of the Loss Function with respect to the parameter of interest.


<div>
<img src="files/gradient_descent_1.png" width="70%" align='center' source="https://github.com/MoinDalvs/Gradient_Descent_For_beginners" />
</div>

**Step One**

Initialize a random parameter value, say $\beta = 0$
 
Calculate the derivative of the Loss Function at that point : $\frac{\delta SSR}{\delta \beta_0}$


<div>
<img src="files/gradient_descent_2.png" width="40%" align='center' source="https://github.com/MoinDalvs/Gradient_Descent_For_beginners" />
</div>

**Step Two**

Move in the opposite direction of the derivative by one step.

<div>
<img src="files/gradient_descent_3.png" width="80%" align='center' source="https://github.com/MoinDalvs/Gradient_Descent_For_beginners" />
</div>

**Note:**

- The step size is proportional to the derivative's value
- It moves according to a chosen Learning Rate = $\eta$

$$\beta^{(1)}_0 = 0 - \eta \frac{\partial L}{\partial \beta_0} (0)$$

The $(1)$ above $\beta$ meaning its our first "epoch" (an iteration). 


The updated intercept value is plugged back into the derivative of the Loss Function, and we repeat the process.

<div>
<img src="files/gradient_descent_4.png" width="80%" align='center' source="https://github.com/MoinDalvs/Gradient_Descent_For_beginners" />
</div>

As the loss approaches its minimum, the derivative gets smaller, and so do the steps.
<div>
<img src="files/gradient_descent_5.png" width="80%" align='center' source="https://github.com/MoinDalvs/Gradient_Descent_For_beginners" />
</div>

This makes the Gradient Descent computationally efficient. It does few calculations far away from the minimum, and more calculations as it approaches the minimum of the Loss Function.

<div>
<img src="files/gradient_descent_6.png" width="80%" align='center' source="https://github.com/MoinDalvs/Gradient_Descent_For_beginners" />
</div>

**When does it stop?**
The Gradient Descent algorithm can have different stopping criteria:

- Minimum Step Size (e.g. 0.001). When the step size is smaller than this threshold, the Gradient Descent has converged, and the corresponding intercept is the optimal value
- Maximum Number of steps (e.g. 1000)


In [ ]:
X = df['weight']
y = df['height']

b1 = 0.64
eta = 0.1 # Learning Rate

# Hypothesis function h
def h(X, b0):
    return b0 + b1 * X

# Initialize intercept at 0 for this example
b0_epoch0 = 0

# L(b0_epoch_0)
np.sum((y - h(X, b0_epoch0)) ** 2)

In [ ]:
# Step 1: compute the derivative of the Loss Function at b0_epoch_0
derivative = np.sum(-2 * (y - h(X, b0_epoch0)))
derivative

In [ ]:
# Step 2: update the intercept
b0_epoch1 = b0_epoch0 - (eta * derivative)
b0_epoch1

We've just done **one epoch**! Let's do an other one!

In [ ]:
# Step1: compute the new derivative at b0_epoch1
derivative = np.sum(-2 * (y - h(X, b0_epoch1)))

# Step2: update the previously updated intercept
b0_epoch2 = b0_epoch1 - eta * derivative
b0_epoch2

<div>
<img src="files/gradient_descent_9.png" width="40%" align='center' source="https://github.com/MoinDalvs/Gradient_Descent_For_beginners" />
</div>

Keep going until it converges to the minimum!

You can also visualize it this way:

<div>
<img src="files/gradient_descent_7.webp" width="55%" align='center' source="https://ucanalytics.com/blogs/intuitive-machine-learning-gradient-descent-simplified/" />
</div>
<br>

This is called the "energy landscape" or the "potential energy surface" of the Loss Function.

Or like this in more complex problems:

<div>
<img src="files/gradient_descent_8.webp" width="60%" align='center' source="https://towardsdatascience.com/its-only-natural-an-excessively-deep-dive-into-natural-gradient-optimization-75d464b89dbb" />
</div>

## What is the effect of learning rate $\eta$?


<div>
<img src="files/gradient_descent_10.png" width="80%" align='center' source="https://github.com/MoinDalvs/Gradient_Descent_For_beginners" />
</div>

**Small Learning Rate**

- A shorter path to the minimum.
- Requires more epochs.
- May get stuck at local minima.

**Large Learning Rate**

- Requires fewer epochs.
- May never converge!

Also the Gradient Descent algorithm always converges faster when **features are scaled**!

> "Without feature scaling, gradient descent will require a lot more steps to reach the minima. In other words, gradient descent will take a lot of time to converge thus increasing the model training time." [source](https://medium.com/analytics-vidhya/why-gradient-descent-doesnt-converge-with-unscaled-features-8b7ed0c8cab6)

## Other solvers

Gradient Descent is computationally expensive on big datasets, couldn't we use less than all observations to compute an "approximate loss"?

- **Mini-Batch Gradient Descent**: At each iteration compute an approximate loss on a mini batch of your data points.
- **Stochastic Gradient Descent (SGD)**: A very common optimisation, that using random mini batch or single data points.


<div>
<img src="files/gradient_descent_11.png" width="70%" align='center' source="https://github.com/MoinDalvs/Gradient_Descent_For_beginners" />
</div>


**Pros**

- SGD is faster for very large datasets.
- Jumps out of local minima!
- Greatly reduces RAM load (useful when you do Deep Learning).

**Cons**

- Needs more epochs.
- Never exactly converges (careful when to stop).
- Maybe slower for small $n$ datasets with many features $p$.
 
To Use when:

- The number of observations in your dataset has 6 digits or more.
- You want to get "un-stuck" from a local minimum.

### Sklearn SGDRegressor and SGDClassifier

- SGDRegressor is a Linear Model (Linear Regression) that uses the Stochastic Gradient Descent as a solver to minimize its Loss Function (MSE)
- SGDClassifier is a Linear Model (Logistic Regression) that uses the Stochastic Gradient Descent as a solver to minimize its Loss Function (Log Loss)

In [ ]:
from sklearn.linear_model import SGDRegressor, LinearRegression

lin_reg = LinearRegression() # OLS solved by matrix inversion (SVD method)
lin_reg_sgd = SGDRegressor(loss='squared_error') # OLS solved by SGD

In [ ]:
from sklearn.datasets import make_regression

# Create a "fake problem" to solve
X, y = make_regression(n_samples=10_000, n_features=2000)

In [ ]:
%%time
lin_reg.fit(X,y)

In [ ]:
%%time
lin_reg_sgd.fit(X,y)

# Model Tuning

For a given model, we have:

- **Parameters** (computing automatically during the fitting process by minimizing $L(\beta)$).

- **Hyperparameters** (Loss function, loss parameters, solver, model specifities etc.)

## Improving model performances


### Solutions for Overfitting

In Machine Learning, most of the time we're dealing with overfitting. On order to reduce overfitting and allow the model to generalize better we can :

- Get more observations (data)
- Feature selection (manual or automated)
- Dimensionality reduction (Unsupervised Learning)
- Early stopping (Deep Learning)
- **Regularization** of your Loss function

## Model Complexity

Let's remember Linear Regression:

$Y = \beta_0 + \beta_1 X_1 + \epsilon$

What about non-linear behavior as below?

<div>
<img src="files/linear_regression_vs_non_linear_regression.png" width="50%" align='center'/>
</div>

If we add a new transformed feature $X^2_1$ we have a better fit (in red)

$Y = \beta_0 + \beta_1 X_1 + \beta_2 X^2_1 + \ldots + \epsilon$

We could engineer very complex features: $X^3, X^6, (X^2_1 * X^5_2), \ldots$

**An interesting property** : our $R^2$, on the train set, will keep increasing with every additional feature! Because it will have more "tools" to fit all the data points.

## Regularization

Most of the times we're fighting against the overfitting phenomenon. To do this we can use various techniques such as **regularization**.

Regularization means adding a penalty term to the Loss that increases with
$\beta$:

$Regularized Loss = Loss(X, y, \beta) + Penalty(\beta)$

Usually we're trying to minimize the loss. But we're adding a penalty which is an increasing function of $\beta$, meaning that the greater are our coefficients, the stronger is the penalty.

In other words, when we're trying to **minimize the loss**, there's a new constraint which is **not having too big coefficients**.

- It forces the model to **shrink certains coefficients** or even **select less features**.
- So it prevents **overfitting**.

$\hat{y} = \beta_0 + \beta_1 X_1 + \beta_2 X^2_1 + \beta_3 X^3_1 + \ldots$

A model with smaller coefficients ($\beta_1$, $\beta_2$ etc.), or even equal to zeros, is a **more simple model**. Instead of taking all the features and trying to link them with all the other ones, our model will focus more on basic relationships, thus **avoiding modelizing the noise**.

## Ridge (L2) and Lasso (L1)

### Formulas

The two most famous Regularization penalties are the Ridge and the Lasso:

$Ridge(X, y, \beta) = \text{Loss}(X, y, \beta) + \alpha \sum\limits_{i=1}^{n} \beta_i^2$

With the ridge the penalty is the sum of the squares of the $\beta$ coefficients.

$Lasso(X, y, \beta) = \text{Loss}(X, y, \beta) + \alpha \sum\limits_{i=1}^{n} |\beta_i|$

With the lasso the penalty is the sum of the absolute values of the $\beta$ coefficients.

### New hyper-paramater : $\alpha$

- It dictates how much the model is regularized.
- A large $\alpha$ forces model complexity to decrease, so variance will be lowered and biais will be greater.

### Ridge or Lasso?



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y=True, as_frame=True)
X.head()

### Documentation

[On sklearn website](https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset)

- **age:** age in years
- **sex:** male of female
- **bmi:** body mass index
- **bp:**  average blood pressure
- **s1:** tc, total serum cholesterol
- **s2:** ldl, low-density lipoproteins
- **s3:** hdl, high-density lipoproteins
- **s4:** tch, total cholesterol / HDL
- **s5:** ltg, possibly log of serum triglycerides level
- **s6:** glu, blood sugar level

**Always scale your features before using regularization!** Here all values have already been scaled.

The **target** variable is a quantitative measure of disease progression one year after baseline.

In [ ]:
y.head()

In sklearn Ridge and Lasso are LinearRegression with a Ridge, or a Lasso, already implemented. You'll notice the $\alpha$ coefficient.

In [ ]:
from sklearn.linear_model import Ridge, Lasso, LinearRegression

# Creating 3 models
linreg = LinearRegression().fit(X, y)
ridge = Ridge(alpha=0.2).fit(X, y)
lasso = Lasso(alpha=0.2).fit(X, y)

coefs = pd.DataFrame({
        "coef_linreg": pd.Series(linreg.coef_, index=X.columns),
        "coef_ridge": pd.Series(ridge.coef_, index=X.columns),
        "coef_lasso": pd.Series(lasso.coef_, index= X.columns)})

coefs.map(lambda x: int(x)).style.map(lambda x: 'color: red' if x == 0 else 'color: black')

The Linear Regression classic says that the disease progression is $-10 * age + (-239 * sex)$ etc.
Some $\beta$ are very large.

- The ridge lowers all of our coef.
- The lasso also lowers our coef and set to zeros some of them, even though they were the highest coefficient.

Increasing the alpha parameter increasing the action of ridge and lasso.

The features which are penalized are most of the time the ones which have the more noise.

In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True, as_frame=True)

def get_coef(model, alpha, X=X, y=y): return model(alpha=alpha).fit(X, y).coef_.round(2)

def get_df(model, alpha_list, X=X, y=y):
    _df = pd.DataFrame()
    for alpha in alpha_list:
        _df = pd.concat([_df, pd.DataFrame([np.append(get_coef(model, alpha=alpha), alpha)], columns=[col for col in X.columns] + ['alpha'])], axis=0)
    return _df

def plot_against_alpha(_df, alpha_name='alpha'):
    fig, ax = plt.subplots(figsize=(10, 6))
    for column in _df.drop(columns=alpha_name):
        ax.plot(_df[alpha_name], _df[column], label=column)
    ax.set_xlabel('Alpha')
    ax.set_ylabel('Coef')
    ax.legend()
    plt.show()

In [ ]:
alpha_list = np.linspace(0.1, 4, 20)
ridge_df = get_df(Ridge, alpha_list)
lasso_df = get_df(Lasso, alpha_list)

#### Ridge

In [ ]:
plot_against_alpha(ridge_df)

#### Lasso

In [ ]:
plot_against_alpha(lasso_df)

- Increasing $\alpha$ in Ridge will only shrink parameters towards zero. As seen above.
- Whereas increasing $\alpha$ in Lasso can shrink parameter to 0 (natural feature selector). As you can see below.

### An other way to see it

<div>
<img src="files/lasso_vs_ridge.png" width="75%" source='https://www.linkedin.com/pulse/ridge-lasso-regularization-gurumaheswara-reddy/' align='center'/>
</div>

"Regularization restricts the allowed positions of β̂ to the blue constraint region:

- For **lasso**, this region is a diamond-shaped because it constrains the absolute value of the coefficients.

- For **ridge**, this region is a circle-shaped because it constrains the square of the coefficients.

The size of the blue region is determined by α, with a smaller α resulting in a larger region
When α is zero, the blue region is infinitely large, and thus the coefficient sizes are not constrained.
When α increases, the blue region gets smaller and smaller."

[source](https://www.linkedin.com/pulse/ridge-lasso-regularization-gurumaheswara-reddy/)

We're trying to minimize the loss function (red circles), from the optimal $\hat\beta$ but with the constraint that the sum of $\beta^2_i$ (Ridge) or $|\beta_i|$ (Lasso) is inferior to a constant.

### Evolution of the Ridge Regularization whith different $\alpha$ values
<div>
<img src="files/ridge_and_alpha.png" width="75%" source='https://www.analyticsvidhya.com/blog/2016/01/ridge-lasso-regression-python-complete-tutorial/' align='center'/>
</div>

For Ridge, the importance of all coefficients is reduced.

### Evolution of the Lasso Regularization whith different $\alpha$ values
<div>
<img src="files/lasso_and_alpha.png" width="75%" source='https://www.analyticsvidhya.com/blog/2016/01/ridge-lasso-regression-python-complete-tutorial/' align='center'/>
</div>

The higher $\alpha$ is, the more features are being cut out. For alpha = 0.01, there's only one feature left, and from $\alpha$ = 1, there's no features left, only the intercept.

## ElasticNet = Lasso + Ridge

ElasticNet = Lasso & Ridge Weighted Average

$L = ||y - \hat{y}||^2 + \alpha (\lambda |\beta| + (1 - \lambda)||\beta||^2)$

Now, we've got 2 hyper-parameters to fine-tune : $\alpha$ and $\lambda$.
If $\lambda$ = 1, it's going to output the same result than lasso. If it's close from zero, it's going to output the same result than Ridge. 



<div>
<img src="files/elasticnet.webp" width="65%" source='https://corporatefinanceinstitute.com/resources/data-science/elastic-net/' align='center'/>
</div>

## Conclusions

- Regularize when you think you are overfitting (learning curves not converging).
- **Ridge** when you believe **all coefficients** may have an impact.
- **Lasso** as a **feature selection tool** and for **better interpretability**.
- Use **ElasticNet** if you need a bit of both.

## How do we find the best hyper-parameters?

### The Grid Search Method

- Hold out a validation set (never use the test set for model tuning!)
- Select which grid of values of hyper-parameters to try out
- For each combination of values, measure your performance on the validation set
- Select hyper-parameters that produce the best performance

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=1)

# Select hyperparam values to try
alphas = [0.01, 0.1, 1] # L1 + L2 
l1_ratios = [0.2, 0.5, 0.8] # Lamba, otherwise : L1 / L2 ratio. So the higher it is the closer we have some L1 (Lasso)

# create all combinations [(0.01, 0.2), (0.01, 0.5), (...)]
from itertools import product
hyperparams = product(alphas, l1_ratios) 

In [ ]:
# Train and CV-score model for each combination
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

for hyperparam in hyperparams:
    alpha = hyperparam[0]
    l1_ratio = hyperparam[1]

    model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)

    r2 = cross_val_score(model, X_train, y_train, cv=5).mean()

    print(f"alpha: {alpha}, l1_ratio: {l1_ratio},   r2: {r2}")

There's a function already in sklearn.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

# Instantiate model
model = ElasticNet()

# Hyperparameter Grid
grid = {
    'alpha': [0.01, 0.1, 1], 
    'l1_ratio': [0.2, 0.5, 0.8] # lambda (remember "L1" is also called "Lasso")
}

# Instantiate Grid Search
search = GridSearchCV(
    model,
    grid, # dictionary of hyperparameter 
    scoring='r2', # or an other Metric for which we want to optimize the regulation
    cv=5, # So validation set is 20% each time, then we get the mean.
    n_jobs=-1, # parallelize computation
    verbose=1,
) 

# Fit data to Grid Search
search.fit(X_train, y_train);

In [ ]:
# Best score
search.best_score_ # a numpy float64 object

In [ ]:
# Best Params
search.best_params_ # dict object

In [ ]:
# Best estimator
search.best_estimator_ # a sklearn object

### Limitations of Grid Search:

- Computationally costly
- The optimal hyperparameter value can be missed

## Random Search

Instead of using a grid of parameter, we can aske sklearn to try random values. You can pass a list of random values or a distribution.


<div>
<img src="files/grid_search_vs_random_search.webp" width="55%" source='https://medium.com/@cjl2fv/an-intro-to-hyper-parameter-optimization-using-grid-search-and-random-search-d73b9834ca0a' align='center'/>
</div>


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

# Instantiate model
model = ElasticNet()

# Hyperparameter Grid
grid = {'l1_ratio': stats.uniform(0, 1),
        'alpha': [0.001, 0.01, 0.1, 1]} # you can pass functions or list of values

# Instantiate Grid Search
search = RandomizedSearchCV(
         model,
         grid, 
         scoring='r2',
         n_iter=100,  # number of cv trainings
         cv=5, # number of folds for each cv
         n_jobs=-1
    ) 

# Fit data to Grid Search
search.fit(X_train, y_train)
search.best_estimator_

In [ ]:
# Generate random numbers from a uniform distribution
data = stats.uniform.rvs(0, 1, size=1000) # rvs = random variates

plt.hist(data, bins=50, density=True, alpha=0.6, color='g')

# Plot the probability density function (PDF)
x = np.linspace(0, 1, 100)
plt.plot(x, stats.uniform.pdf(x, 0, 1), 'r-', lw=2, label='Uniform PDF')
plt.xlabel('Value')
plt.ylabel('Probability Density')
plt.title('Uniform Distribution')
plt.legend()

plt.show()

In [ ]:
plt.figure(figsize=(5, 3))

# dist = stats.norm(10, 2) # if you have a best guess (say: 10)
# dist = stats.randint(1,100) # if you have no idea
# dist = stats.uniform(1, 100) # same
# dist = stats.loguniform(0.01, 1) # Coarse grain search

r = dist.rvs(size=10000) # Random draws
plt.hist(r);

### RandomizedSearch vs GridSearch

#### GridSearch

- You want to explore a specific hyperparameter or combination.
- You are optimizing few parameters.
- You think all hyperparameters are equally important for performance.

#### Randomized Search

- Less typing, if you want to try many different values.
- Control for the number of combinations to try and time spent searching.
- Useful when some hyper-parameters are more important than others.

#### Use Both

You can actually **start with a random search** if you're not sure of what're you're looking for, then use a **grid search** to test all the combinations between the values you have found.